# 🎯 Comparing Epsilon Values in Epsilon-Greedy

**ผู้เขียน**: Hamdee Naseng  
**วันที่**: 3 พฤศจิกายน 2025  
**วัตถุประสงค์**: เปรียบเทียบค่า ε (epsilon) ต่าง ๆ ใน Epsilon-Greedy Algorithm

---

## 🎯 โค้ดนี้ทำอะไร?

### 📊 Epsilon-Greedy Algorithm

**คำถาม**: ค่า ε (epsilon) ไหนดีที่สุดสำหรับ Multi-Armed Bandit?

**ตัวเลือก**:
- **ε = 0.1** (10% explore, 90% exploit)
- **ε = 0.05** (5% explore, 95% exploit)
- **ε = 0.01** (1% explore, 99% exploit)

**สิ่งที่จะเปรียบเทียบ**:
- Cumulative Average Reward
- จำนวนครั้งที่เลือก suboptimal arm
- ความเร็วในการ converge

---

## 🔑 Key Concepts:

### 1. **Epsilon-Greedy Strategy**
```
ในแต่ละรอบ:
  ถ้า random() < ε:
    Explore: เลือก arm แบบสุ่ม
  ถ้าไม่:
    Exploit: เลือก arm ที่มี estimate ดีที่สุด
```

### 2. **Trade-off: Exploration vs Exploitation**
- **ε สูง (0.1)**: Explore มาก → เรียนรู้เร็ว แต่ waste opportunities
- **ε ต่ำ (0.01)**: Exploit มาก → ได้ reward มาก แต่อาจติดใน local optimum
- **ε กลาง (0.05)**: Balance ระหว่างสองแบบ

### 3. **Normal Distribution Bandits**
- แต่ละ arm มี reward ~ N(μ, 1)
- μ₁ = 1.5, μ₂ = 2.5, μ₃ = 3.5
- Arm 3 คือ optimal arm

---

## 🚀 สิ่งที่จะได้เรียนรู้:

1. ✅ วิธีทำงานของ Epsilon-Greedy
2. ✅ ผลกระทบของค่า ε ต่าง ๆ
3. ✅ การเปรียบเทียบ performance
4. ✅ การเลือกค่า ε ที่เหมาะสม

---

## 📦 Step 1: Import และตั้งค่า

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ตั้งค่า font สำหรับภาษาไทย
import matplotlib.font_manager as fm

font_path = '../../font/Prompt/Prompt-Regular.ttf'
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
font_name = font_prop.get_name()

plt.rcParams['font.family'] = font_name
plt.rcParams['axes.unicode_minus'] = False

print("✅ Import สำเร็จ!")
print(f"✅ ตั้งค่า Font ภาษาไทยเป็น '{font_name}' เรียบร้อย")

## 🎰 Step 2: สร้าง BanditArm Class

Class นี้จำลอง Bandit Arm ที่ให้ reward จาก **Normal Distribution**

In [ ]:
class BanditArm:
    """
    Bandit Arm ที่ให้ reward จาก Normal Distribution
    
    reward ~ N(m, 1)
    - m: ค่าเฉลี่ยจริงของ arm
    - variance = 1 (คงที่)
    """
    def __init__(self, m):
        self.m = m                # ค่าเฉลี่ยจริง (true mean)
        self.m_estimate = 0       # ค่าประมาณที่เรารู้
        self.N = 0                # จำนวนครั้งที่ถูก pull
    
    def pull(self):
        """Pull arm และได้ reward"""
        # reward = N(m, 1)
        return np.random.randn() + self.m
    
    def update(self, x):
        """อัพเดทค่าประมาณด้วย reward ใหม่
        
        ใช้ incremental mean:
        new_mean = old_mean + (1/N) * (new_value - old_mean)
        
        หรือเขียนเป็น:
        new_mean = (1 - 1/N) * old_mean + (1/N) * new_value
        """
        self.N += 1
        self.m_estimate = (1 - 1.0/self.N) * self.m_estimate + 1.0/self.N * x
    
    def __repr__(self):
        return f"BanditArm(m={self.m:.2f}, estimate={self.m_estimate:.2f}, N={self.N})"

print("✅ สร้าง BanditArm class เรียบร้อย!")
print("\n💡 BanditArm จะ:")
print("   1. Pull → ได้ reward จาก N(m, 1)")
print("   2. Update → อัพเดทค่าประมาณด้วย incremental mean")
print("   3. ไม่ใช้ Beta distribution (เพราะ reward เป็น continuous)")

## 🧪 Step 3: ทดสอบ BanditArm

มาดูว่า BanditArm ทำงานอย่างไร

In [ ]:
# สร้าง arm ทดสอบ (true mean = 2.5)
test_arm = BanditArm(2.5)

print("🎰 ทดสอบ BanditArm (true mean = 2.5)\n")
print(f"เริ่มต้น: {test_arm}\n")

# Pull 5 ครั้ง
print("Pull 5 ครั้ง:")
for i in range(5):
    reward = test_arm.pull()
    test_arm.update(reward)
    print(f"  Round {i+1}: reward = {reward:.2f}, estimate = {test_arm.m_estimate:.2f}")

print(f"\n📊 หลัง 5 ครั้ง: {test_arm}")

# Pull อีก 95 ครั้ง
for i in range(95):
    reward = test_arm.pull()
    test_arm.update(reward)

print(f"📊 หลัง 100 ครั้ง: {test_arm}")

print("\n💡 สังเกต:")
print("  - เริ่มต้น estimate = 0")
print("  - หลังจาก pull หลาย ๆ ครั้g → estimate เข้าใกล้ true mean (2.5)")
print("  - Variance ของ reward ทำให้ estimate ไม่ตรงทุกครั้ง")

## 🏃 Step 4: สร้างฟังก์ชัน run_experiment

ฟังก์ชันนี้จะรัน Epsilon-Greedy experiment

In [ ]:
def run_experiment(m1, m2, m3, eps, N):
    """
    รัน Epsilon-Greedy experiment
    
    Parameters:
    -----------
    m1, m2, m3 : float
        True means ของแต่ละ arm
    eps : float
        Epsilon (ความน่าจะเป็นของการ explore)
    N : int
        จำนวนรอบ
    
    Returns:
    --------
    cumulative_average : array
        Average reward เมื่อเวลาผ่านไป
    """
    # สร้าง 3 bandits
    bandits = [BanditArm(m1), BanditArm(m2), BanditArm(m3)]
    
    # หา optimal arm
    means = np.array([m1, m2, m3])
    true_best = np.argmax(means)  # arm ที่ดีที่สุด
    count_suboptimal = 0          # นับครั้งที่เลือกผิด
    
    # เก็บ rewards
    data = np.empty(N)
    
    print(f"\n🎯 เริ่มการทดลอง (ε = {eps}, N = {N:,})")
    print(f"   Arm means: {means}")
    print(f"   Optimal arm: Arm {true_best + 1} (mean = {means[true_best]:.1f})")
    print(f"   Strategy: {eps:.0%} explore, {(1-eps):.0%} exploit\n")
    
    for i in range(N):
        # Epsilon-Greedy
        p = np.random.random()
        
        if p < eps:
            # Explore: เลือกแบบสุ่ม
            j = np.random.choice(len(bandits))
        else:
            # Exploit: เลือก arm ที่มี estimate สูงสุด
            j = np.argmax([b.m_estimate for b in bandits])
        
        # Pull arm ที่เลือก
        x = bandits[j].pull()
        bandits[j].update(x)
        
        # นับครั้งที่เลือกผิด
        if j != true_best:
            count_suboptimal += 1
        
        # เก็บข้อมูล
        data[i] = x
    
    # คำนวณ cumulative average
    cumulative_average = np.cumsum(data) / (np.arange(N) + 1)
    
    # แสดงผลลัพธ์
    print("📊 ผลลัพธ์:")
    for i, b in enumerate(bandits):
        is_best = " ⭐" if i == true_best else ""
        print(f"   Arm {i+1}: true={b.m:.1f}, estimate={b.m_estimate:.3f}, pulls={b.N:,}{is_best}")
    
    suboptimal_pct = float(count_suboptimal) / N
    print(f"\n   Suboptimal choices: {count_suboptimal:,} / {N:,} ({suboptimal_pct:.1%})")
    print(f"   Final average reward: {cumulative_average[-1]:.3f}")
    
    # วาดกราฟ
    plt.figure(figsize=(12, 6))
    
    plt.plot(cumulative_average, linewidth=2, label=f'ε = {eps}')
    plt.axhline(m1, color='blue', linestyle='--', alpha=0.5, label=f'Arm 1: μ={m1}')
    plt.axhline(m2, color='orange', linestyle='--', alpha=0.5, label=f'Arm 2: μ={m2}')
    plt.axhline(m3, color='green', linestyle='--', alpha=0.5, label=f'Arm 3: μ={m3}')
    
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Cumulative Average Reward', fontsize=12)
    plt.title(f'Epsilon-Greedy (ε = {eps})', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(alpha=0.3)
    plt.xscale('log')
    plt.tight_layout()
    plt.show()
    
    return cumulative_average

print("✅ สร้างฟังก์ชัน run_experiment เรียบร้อย!")

## 🧪 Step 5: ทดลองกับ ε = 0.1 (10% explore)

มาดูว่า ε = 0.1 ทำงานอย่างไร

In [ ]:
# ตั้งค่า
m1, m2, m3 = 1.5, 2.5, 3.5  # True means
N = 100000                   # จำนวนรอบ

# รัน experiment
c_1 = run_experiment(m1, m2, m3, eps=0.1, N=N)

print("\n💡 สังเกต:")
print("  - ε = 0.1 → Explore 10% ของเวลา")
print("  - เรียนรู้เร็วพอสมควร")
print("  - แต่ยังเลือก suboptimal arms บ่อย (~10%)")

## 🧪 Step 6: ทดลองกับ ε = 0.05 (5% explore)

มาดูว่า ε = 0.05 ทำงานอย่างไร

In [ ]:
# รัน experiment
c_05 = run_experiment(m1, m2, m3, eps=0.05, N=N)

print("\n💡 สังเกต:")
print("  - ε = 0.05 → Explore 5% ของเวลา")
print("  - Exploit มากขึ้น → Average reward สูงขึ้น")
print("  - Suboptimal choices น้อยลง (~5%)")

## 🧪 Step 7: ทดลองกับ ε = 0.01 (1% explore)

มาดูว่า ε = 0.01 ทำงานอย่างไร

In [ ]:
# รัน experiment
c_01 = run_experiment(m1, m2, m3, eps=0.01, N=N)

print("\n💡 สังเกต:")
print("  - ε = 0.01 → Explore 1% ของเวลา")
print("  - Exploit มากที่สุด → Average reward ใกล้ optimal")
print("  - Suboptimal choices น้อยมาก (~1%)")
print("  - แต่เสี่ยงติด local optimum ถ้ายังไม่รู้จัก best arm")

## 📊 Step 8: เปรียบเทียบ Log Scale

เปรียบเทียบทั้ง 3 ค่า ε ในกราฟ log scale

In [ ]:
# วาดกราฟเปรียบเทียบ (log scale)
plt.figure(figsize=(14, 7))

plt.plot(c_1, linewidth=2, label='ε = 0.1 (10% explore)', alpha=0.8)
plt.plot(c_05, linewidth=2, label='ε = 0.05 (5% explore)', alpha=0.8)
plt.plot(c_01, linewidth=2, label='ε = 0.01 (1% explore)', alpha=0.8)

plt.axhline(m3, color='green', linestyle='--', linewidth=2, label=f'Optimal (μ = {m3})')

plt.xlabel('Iteration (Log Scale)', fontsize=12)
plt.ylabel('Cumulative Average Reward', fontsize=12)
plt.title('เปรียบเทียบ Epsilon Values (Log Scale)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.xscale('log')
plt.tight_layout()
plt.show()

print("📊 ความหมายของกราฟ (Log Scale):")
print("\n  - แกน X เป็น log scale → เห็นพัฒนาการช่วงแรก ๆ ชัดเจน")
print("  - ε = 0.1 (สีน้ำเงิน): เริ่มช้า แต่ stable เร็ว")
print("  - ε = 0.05 (สีส้ม): ดีที่สุด → Balance ระหว่าง exploration/exploitation")
print("  - ε = 0.01 (สีเขียว): เริ่มดี แต่ convergence ช้า")

## 📊 Step 9: เปรียบเทียบ Linear Scale

เปรียบเทียบทั้ง 3 ค่า ε ในกราฟ linear scale

In [ ]:
# วาดกราฟเปรียบเทียบ (linear scale)
plt.figure(figsize=(14, 7))

plt.plot(c_1, linewidth=2, label='ε = 0.1 (10% explore)', alpha=0.8)
plt.plot(c_05, linewidth=2, label='ε = 0.05 (5% explore)', alpha=0.8)
plt.plot(c_01, linewidth=2, label='ε = 0.01 (1% explore)', alpha=0.8)

plt.axhline(m3, color='green', linestyle='--', linewidth=2, label=f'Optimal (μ = {m3})')

plt.xlabel('Iteration (Linear Scale)', fontsize=12)
plt.ylabel('Cumulative Average Reward', fontsize=12)
plt.title('เปรียบเทียบ Epsilon Values (Linear Scale)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 ความหมายของกราฟ (Linear Scale):")
print("\n  - แกน X เป็น linear scale → เห็นพฤติกรรมระยะยาว")
print("  - ε = 0.1: Average reward ต่ำกว่าเส้น optimal ชัดเจน")
print("  - ε = 0.05: ใกล้เส้น optimal มากที่สุด ✅")
print("  - ε = 0.01: ค่อนข้างใกล้ optimal แต่ convergence ช้า")

## 📈 Step 10: วิเคราะห์ผลเปรียบเทียบ

มาดูตัวเลขสถิติโดยละเอียด

In [ ]:
# สรุปผลการทดลอง
print("\n" + "="*70)
print("📊 สรุปผลการเปรียบเทียบ Epsilon Values")
print("="*70)

print("\n🎯 ตั้งค่า:")
print(f"   Arm 1: μ = {m1}")
print(f"   Arm 2: μ = {m2}")
print(f"   Arm 3: μ = {m3} ⭐ (Optimal)")
print(f"   จำนวนรอบ: {N:,}")

print("\n📊 Final Cumulative Average Rewards:")
print(f"   ε = 0.10: {c_1[-1]:.4f}")
print(f"   ε = 0.05: {c_05[-1]:.4f} ✅ (ดีที่สุด)")
print(f"   ε = 0.01: {c_01[-1]:.4f}")

# คำนวณ gap จาก optimal
print("\n📉 Gap จาก Optimal (μ = 3.5):")
print(f"   ε = 0.10: {m3 - c_1[-1]:.4f} ({(m3 - c_1[-1])/m3:.1%} loss)")
print(f"   ε = 0.05: {m3 - c_05[-1]:.4f} ({(m3 - c_05[-1])/m3:.1%} loss) ✅")
print(f"   ε = 0.01: {m3 - c_01[-1]:.4f} ({(m3 - c_01[-1])/m3:.1%} loss)")

# คำนวณ cumulative regret
regret_1 = (m3 - c_1).sum()
regret_05 = (m3 - c_05).sum()
regret_01 = (m3 - c_01).sum()

print("\n💸 Cumulative Regret (ยิ่งน้อยยิ่งดี):")
print(f"   ε = 0.10: {regret_1:,.2f}")
print(f"   ε = 0.05: {regret_05:,.2f} ✅ (น้อยที่สุด)")
print(f"   ε = 0.01: {regret_01:,.2f}")

print("\n🎓 ข้อสรุป:")
print(f"   1. ε = 0.05 ให้ผลลัพธ์ดีที่สุด")
print(f"   2. ε = 0.1 explore มากเกินไป → waste opportunities")
print(f"   3. ε = 0.01 explore น้อยเกินไป → convergence ช้า")
print(f"   4. ต้องเลือก ε ที่ balance ระหว่าง exploration/exploitation")

print("\n" + "="*70)

## 🎓 สรุป: สิ่งที่เราได้เรียนรู้

### 🔑 แนวคิดหลัก (Key Concepts):

#### 1. **Epsilon-Greedy Algorithm**

**สูตร**:
```
ในแต่ละรอบ:
  random_value = random()
  
  if random_value < ε:
    # Explore
    action = random_choice(arms)
  else:
    # Exploit
    action = argmax(estimated_values)
```

**พารามิเตอร์**:
- **ε (epsilon)**: ความน่าจะเป็นของการ explore (0 ≤ ε ≤ 1)

#### 2. **Normal Distribution Bandits**

**ความแตกต่างจาก Beta-Bernoulli**:
- **Beta-Bernoulli**: Reward = 0 หรือ 1 (click/no-click)
- **Normal**: Reward เป็น continuous value ~ N(μ, σ²)

**Update Rule**:
```
Incremental Mean:
new_estimate = old_estimate + (1/N) × (reward - old_estimate)

หรือ:
new_estimate = (1 - 1/N) × old_estimate + (1/N) × reward
```

**ข้อดี**:
- ใช้ได้กับ reward ที่เป็นจำนวนจริง (เงิน, ระยะเวลา, คะแนน)
- Update ง่าย ไม่ต้องเก็บข้อมูลทั้งหมด

#### 3. **ผลของค่า ε ต่าง ๆ**

| ε | Explore % | Exploit % | ข้อดี | ข้อเสีย | เหมาะกับ |
|---|-----------|-----------|-------|---------|----------|
| **0.1** | 10% | 90% | เรียนรู้เร็ว | Waste 10% | ไม่แน่ใจว่า arms ไหนดี |
| **0.05** | 5% | 95% | **Balance ดีที่สุด** | - | **แนะนำ** ✅ |
| **0.01** | 1% | 99% | Exploit มาก | Convergence ช้า | รู้แน่ว่า arm ไหนดี |

#### 4. **Exploration vs Exploitation Trade-off**

**Exploration** (สำรวจ):
- ✅ ค้นหา arms ที่ดีกว่า
- ✅ เรียนรู้ environment
- ❌ อาจได้ reward ต่ำในระยะสั้น

**Exploitation** (ใช้ประโยชน์):
- ✅ ได้ reward สูงในระยะสั้น
- ❌ อาจพลาด optimal arm
- ❌ ติดใน local optimum

**Balance**:
- เริ่มต้น: Explore มาก (ε สูง)
- เมื่อรู้แล้ว: Exploit มาก (ε ต่ำ)
- **Adaptive ε**: ε = 1/t หรือ ε = 1/√t

#### 5. **เปรียบเทียบกับ Thompson Sampling**

| Aspect | Epsilon-Greedy | Thompson Sampling |
|--------|----------------|-------------------|
| **Philosophy** | Fixed exploration rate | Probability matching |
| **Implementation** | ง่ายมาก | ซับซ้อนหน่อย |
| **Performance** | ดี (ถ้าตั้ง ε ถูก) | ดีกว่า (optimal) |
| **Adaptation** | ต้องปรับ ε เอง | Auto-adaptive |
| **Parameter tuning** | ต้องหา ε ที่ดี | ไม่ต้อง |

### 📊 ผลการทดลอง:

**สิ่งที่เห็น**:
1. ✅ **ε = 0.05 ดีที่สุด** → Cumulative reward สูงสุด, regret ต่ำสุด
2. ✅ ε = 0.1 explore มากเกินไป → Loss ~3%
3. ✅ ε = 0.01 exploit มากเกินไป → Convergence ช้า
4. ✅ Log scale แสดงพัฒนาการช่วงแรก, Linear scale แสดงระยะยาว

### 🆚 Comparison with Other Algorithms:

| Algorithm | Regret Bound | Implementation | Best For |
|-----------|--------------|----------------|----------|
| **Random** | Linear | ง่ายที่สุด | Baseline |
| **Greedy** | Linear | ง่าย | ไม่ควรใช้ |
| **ε-Greedy** | O(log N) | ง่าย | Simple problems |
| **UCB** | O(log N) | ปานกลาง | Theoretical optimal |
| **Thompson** | O(log N) | ซับซ้อน | **Best overall** ✅ |

### 🌍 ประยุกต์ใช้:

1. **A/B Testing**:
   - ทดสอบ web design, features
   - ε = 0.05 เหมาะกับ traffic ปานกลาง

2. **Online Advertising**:
   - เลือก ads ที่ดีที่สุด
   - ε-Greedy ง่ายต่อการ implement

3. **Recommendation Systems**:
   - แนะนำ products, content
   - Adaptive ε ตามเวลา

4. **Resource Allocation**:
   - เลือก server, routing
   - ε ต่ำ (exploit มาก)

### 💡 Key Takeaways:

1. **ε = 0.05 เหมาะสมสำหรับหลาย ๆ กรณี** (5% explore)
2. **Trade-off สำคัญ**: Explore มาก = เรียนรู้เร็ว แต่ waste, Exploit มาก = ได้ reward แต่อาจพลาด optimal
3. **Adaptive ε ดีกว่า Fixed**: ε = 1/t หรือ ε = 1/√t
4. **Thompson Sampling ดีกว่า** แต่ซับซ้อนกว่า
5. **Normal distribution bandits**: ใช้กับ continuous rewards (เงิน, เวลา, คะแนน)

### 🔧 Improvements:

**Adaptive Epsilon**:
```python
# ε ลดลงเมื่อเวลาผ่านไป
eps = 1.0 / (t + 1)        # 1/t decay
eps = 1.0 / np.sqrt(t + 1) # 1/√t decay
```

**UCB (Upper Confidence Bound)**:
```python
# เลือก arm ที่มี UCB สูงสุด
ucb = mean_estimate + np.sqrt(2 * np.log(total_pulls) / arm_pulls)
```

**Thompson Sampling**:
```python
# Sample จาก posterior แล้วเลือก arm ที่มีค่าสูงสุด
sample = np.random.normal(mean_estimate, std_estimate)
```

### 📚 Incremental Mean Formula:

**ทำไมใช้สูตรนี้?**
```
new_mean = old_mean + (1/N) × (new_value - old_mean)
```

**ข้อดี**:
- ✅ ไม่ต้องเก็บข้อมูลทั้งหมด (Memory efficient)
- ✅ คำนวณเร็ว (O(1) per update)
- ✅ Numerically stable

**Proof**:
```
new_mean = (sum + new_value) / (N + 1)
         = (N × old_mean + new_value) / (N + 1)
         = N/(N+1) × old_mean + 1/(N+1) × new_value
         = old_mean + 1/(N+1) × (new_value - old_mean)
```

---

## 🎉 ยินดีด้วย!

คุณเข้าใจ **Epsilon-Greedy Algorithm** และ **การเปรียบเทียบค่า ε** แล้ว!

**Next Steps**:
- ทดลอง **Adaptive Epsilon** (ε ลดลงเมื่อเวลาผ่านไป)
- เปรียบเทียบกับ **UCB (Upper Confidence Bound)**
- ศึกษา **Thompson Sampling** (ดีกว่า ε-Greedy)
- ใช้กับ **Real-world datasets**

---